# Using Geopandas to Read and Write Vector Data

<img width="20%" src="https://geopandas.readthedocs.io/en/latest/_static/geopandas_logo_web.svg"></img>


GeoPandas is a popular Python library designed to simplify working with geospatial data in Python by extending the capabilities of the pandas library. It provides data structures and functions needed to manipulate and analyze geospatial data, such as points, lines, and polygons, and to perform various spatial operations, like spatial joins, overlays, and projections.

GeoPandas builds upon several core Python libraries, including pandas, Shapely, Fiona, and pyproj. These dependencies provide the underlying functionality for handling geospatial data structures, file I/O, and coordinate transformations.

http://geopandas.readthedocs.io




We cam import geopandas. Most developers import it as "gpd" to type less.

In [ ]:
import geopandas as gpd

## Example 1: Reading Natural Earth Dataset

In the previous lecture we downloaded the natural earth vector dataset and looked at the airports and countries datasets. 

Let's do that again but this time using GeoPandas

In [ ]:
filename = "geodata/packages/natural_earth_vector.gpkg"

airports = gpd.read_file(filename, layer="ne_10m_airports")

the variable "airports" is a "geopandas data frame". We can just display it in jupyter lab:

The last column is "geometry" and contains the geometry. Compared to fiona this is really easy!

In [ ]:
airports

There are 41 columns and 893 rows. The geodataframe has an attribute "shape", where we can also get this information:

In [ ]:
airports.shape

We can create a new dataframe with less rows by just telling which rows we want. We should always keep the **geometry** row.

In [ ]:
airports2 = airports[['scalerank', 'type', 'name','iata_code', 'geometry']] 

In [ ]:
airports2

we can also call .head(n) to display the first n entries:

In [ ]:
airports2.head(10)

### Sorting

Sorting is very easy. You must specify which column is sorted

In [ ]:
airports2.sort_values(by="name", ascending=True)

### Queries

We can do some queries:

In [ ]:
airports2.query("scalerank == 2")

In [ ]:
airports2.query("iata_code == 'ZRH'  ")

## Example 2: Creating a Geopandas dataframe manually and exporting it

Let's assume we have some data and want to create a geopandas dataframe from scratch.

We have a list of mountain peaks in Switzerland in the format [latitude, longitude, name, elevation]

In [ ]:
data = [[45.922513343092916, 7.835574679184418,'Liskamm',4527],
[45.941997570720375, 7.869820276613906,'Nordend',4609],
[46.10902325837147, 7.863895545667632,'Nadelhorn',4327],
[45.932186337151684, 7.8714190183674555,'Zumsteinspitze',4563],
[46.08336532442726, 7.857296913890337,'Täschhorn',4491],
[45.91669904679932, 7.863563975062021,'Ludwigshöhe',4341],
[45.93756139078208, 7.299279971077615,'Grand Combin de Grafeneire',4314],
[45.922513343092916, 7.835574679184418,'Lyskamm',4527],
[45.93683662540408, 7.866814344981748,'Dufourspitze (Pointe Dufour)',4634],
[46.10129664518156, 7.716156885858494,'Weisshorn',4506],
[45.976340506120614, 7.658691510512221,'Monte Cervino',4478],
[45.976340506120614, 7.658691510512221,'Matterhorn',4478],
[45.93674004101607, 7.86855410887458,'Grenzgipfel',4618],
[45.92712756883081, 7.876921984235257,'Signalkuppe (Punta Gnifetti)',4554],
[46.093839189553464, 7.858928716434883,'Dom',4545],
[46.107109586833495, 7.711724522200983,'Grand Gendarme',4331],
[45.919638502715564, 7.8711910872756405,'Parrotspitze',4432],
[46.093839189553464, 7.858928716434883,'Mischabel',4545],
[46.03426257063022, 7.61204033560156,'Dent Blanche',4357]]

The first step is we have to convert this list to a columns based approach using dictionary, e.g.


    data_as_dict = {"Longitude": [list of longitudes],
                    "Latitude":  [list of latitudes],
                    "Name": [list of names],
                    "Elevation": [list of elevation values]}
    
    
So lets create an empty dictionary and add data to it:

In [ ]:
data_as_dict = {"Longitude": [],
                "Latitude": [],
                "Name": [],
                "Elevation": []}

for row in data:
    data_as_dict["Latitude"].append(row[0])
    data_as_dict["Longitude"].append(row[1])
    data_as_dict["Name"].append(row[2])
    data_as_dict["Elevation"].append(row[3])

Now we do have a dictionary with data in "columns":

In [ ]:
print(data_as_dict)

Now we can create a pandas (not geopandas) dataframe out of it:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data_as_dict)

In [ ]:
df.head(12)

Isn't that great? The only problem is we don't have a GeoPandas dataframe. To do that we have to know about the Python Module Shapely (which will be introduced in Lecture 11). Shapely comes with Point, LineString, Polygon etc. classes where we can convert the geometry to it.

In [ ]:
from shapely import Point

In [ ]:
data_as_dict_geo = {"Name": [],
                "Elevation": [],
                "geometry": []}

for row in data:
    p = Point(row[1],row[0]) # first longitude, then latitude
    data_as_dict_geo["geometry"].append(p)
    data_as_dict_geo["Name"].append(row[2])
    data_as_dict_geo["Elevation"].append(row[3])

In [ ]:
print(data_as_dict_geo)

Now we can create a GeoDataFrame out of it. The option "crs" specifies the coordinate reference system which is WGS84. (We will learn about that in the next lecture.)

In [ ]:
gdf = gpd.GeoDataFrame(data_as_dict_geo, crs="EPSG:4326")

In [ ]:
gdf.head()

Now we can export the GeoDataframe. We can use any format supported for writing from fiona.

    import fiona
    
    fiona.supported_drivers



In [ ]:
gdf.to_file("geodata/mountainpeaks.shp", driver="ESRI Shapefile")

now we could open it using QGIS...

## Example 3: USGS Earthquake Data

The United States Geological Survey (USGS) provides earthquake data in GeoJSON format through their Earthquake Hazards Program. This data contains information about recent and historical earthquakes, including their locations, magnitudes, depths, and other relevant attributes.

The USGS earthquake data is available through their API, which allows you to query and filter the data based on various criteria, such as time range, magnitude range, and geographic region.

In [ ]:
import requests

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson"

data = requests.get(url)
file = open("geodata/earthquakes.geojson","wb")
file.write(data.content)
file.close()

In [ ]:
quakes = gpd.read_file("geodata/earthquakes.geojson")
quakes.head(3)

In [ ]:
quakes = quakes[["time","mag", "place","geometry"]].copy()
quakes.head()

In [ ]:
quakes.mag.hist(bins=16);

In [ ]:
from datetime import datetime, timezone
datetime.fromtimestamp(1602758052977/1000, timezone.utc) # time in **seconds** since 1.1.1970

In [ ]:
from datetime import datetime, timezone

data = []
for row in range(0,len(quakes)):
    time = quakes.iloc[row].time
    t = str(datetime.fromtimestamp(time/1000.0, timezone.utc))
    data.append(t)


In [ ]:
quakes["time_utc"] = data
quakes.head()

In [ ]:
quakes = quakes.drop(["time"], axis=1)

In [ ]:
quakes.head()

In [ ]:
quakes2 = quakes[["time_utc", "place", "mag", "geometry"]].copy()
quakes2.head()

In [ ]:
quakes.sort_values(["mag"], ascending=False).head()

In [ ]:
quakes.plot();

It would be nice to draw earthquakes on a map. So let's look at that next.

In [ ]:
gdfCountries = gpd.read_file("geodata/packages/natural_earth_vector.gpkg", 
                              layer="ne_110m_admin_0_countries", 
                              encoding="utf-8")

In [ ]:
gdfCountries.head()

In [ ]:
ax = gdfCountries.plot(figsize=(20,10), facecolor="#BBFFBB", edgecolor="#000000")
quakes.plot(ax=ax, color="#005500", markersize=40);